In [53]:
import requests
import pandas as pd
import utils.misc as misc
from config import *
import yahoo_fin.stock_info as si
from yahooquery import Ticker
import quandl
import os
from datetime import datetime, timedelta
from finvizfinance.screener.custom import Custom
import time

In [54]:
BASE_URL = r"https://www.alphavantage.co/query?"
av_api = 'CQLP7J9VZG14Y7O4'
av_api_basil = 'IHLNUQ0G66C0MJID'

using proxies

In [63]:
def av_api(func, api_key, symbol):
    assert symbol is not None
    symbol = symbol.strip().upper()

    url = f"{BASE_URL}function={func}&symbol={symbol}&apikey={api_key}"
    response = requests.get(url)
    if func == 'EARNINGS':
        try:
            output = pd.DataFrame(response.json()['quarterlyEarnings'])
        except:
            print(response.json())
            output = None
    elif func == 'BALANCE_SHEET':
        print(response)
        print(response.json())
        return pd.DataFrame(response.json()['quarterlyReports'])
    elif func == 'INCOME_STATEMENT':
        return pd.DataFrame(response.json()['quarterlyReports'])
    return output

In [56]:
def get_av_data(symbol):
    #For reported Earnings & Analyst Estimated EPS, totalRevenue, commonStockSharesOutstanding
    df1 = av_api('EARNINGS',av_api, symbol)
    print(df1)
    df2 = av_api('INCOME_STATEMENT',av_api, symbol)[['fiscalDateEnding','totalRevenue']]
    print(df2)
    df1 = df1.merge(df2, on='fiscalDateEnding', how='left')
    df3 = av_api('BALANCE_SHEET',av_api, symbol)[['fiscalDateEnding','commonStockSharesOutstanding']]
    df1 = df1.merge(df3, on='fiscalDateEnding', how='left')

    return df1

In [68]:
# %%timeit

success = 0
for i in range(50):
    while True:
        try:
            df1 = av_api('EARNINGS',av_api_basil, 'META')
            print(df1.head(1))
            success+=1
            break
        except:
            time.sleep(0.001)
            continue

  fiscalDateEnding reportedDate reportedEPS estimatedEPS surprise  \
0       2022-12-31   2023-02-01        1.76         2.22    -0.46   

  surprisePercentage  
0           -20.7207  
  fiscalDateEnding reportedDate reportedEPS estimatedEPS surprise  \
0       2022-12-31   2023-02-01        1.76         2.22    -0.46   

  surprisePercentage  
0           -20.7207  
  fiscalDateEnding reportedDate reportedEPS estimatedEPS surprise  \
0       2022-12-31   2023-02-01        1.76         2.22    -0.46   

  surprisePercentage  
0           -20.7207  
  fiscalDateEnding reportedDate reportedEPS estimatedEPS surprise  \
0       2022-12-31   2023-02-01        1.76         2.22    -0.46   

  surprisePercentage  
0           -20.7207  
  fiscalDateEnding reportedDate reportedEPS estimatedEPS surprise  \
0       2022-12-31   2023-02-01        1.76         2.22    -0.46   

  surprisePercentage  
0           -20.7207  
  fiscalDateEnding reportedDate reportedEPS estimatedEPS surprise  \
0     

In [67]:
success

50

In [6]:
def get_ex_dates(symbol):
    symbol = 'META'
    tkr = Ticker(symbol)
    df = tkr.sec_filings
    df = df[(df['type'] == '10-Q') | ((df['type'] == '10-K'))]
    df.reset_index(inplace=True)
    df = df[['date','title']]
    df.rename(columns={'date': 'Ex-Date'}, inplace=True)
    return df

In [37]:
av_api('BALANCE_SHEET', av_api, symbol)

<Response [200]>
{'symbol': 'EXFY', 'annualReports': [{'fiscalDateEnding': '2021-12-31', 'reportedCurrency': 'USD', 'totalAssets': '183213000', 'totalCurrentAssets': '157642000', 'cashAndCashEquivalentsAtCarryingValue': '98398000', 'cashAndShortTermInvestments': '98398000', 'inventory': 'None', 'currentNetReceivables': '21139000', 'totalNonCurrentAssets': '19259000', 'propertyPlantEquipment': '15930000', 'accumulatedDepreciationAmortizationPPE': '5984000', 'intangibleAssets': '6359000', 'intangibleAssetsExcludingGoodwill': '6359000', 'goodwill': 'None', 'investments': 'None', 'longTermInvestments': 'None', 'shortTermInvestments': 'None', 'otherCurrentAssets': '14201000', 'otherNonCurrentAssets': '710000', 'totalLiabilities': '106598000', 'totalCurrentLiabilities': '53576000', 'currentAccountsPayable': '3752000', 'deferredRevenue': '200000', 'currentDebt': '1342000', 'shortTermDebt': 'None', 'totalNonCurrentLiabilities': '53222000', 'capitalLeaseObligations': '473000', 'longTermDebt': '

,fiscalDateEnding,reportedCurrency,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,...,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding
0,2022-09-30,USD,211693000,188790000,106212000,106212000,None,21071000,16761000,14872000,...,549000,51572000,112162000,42106000,1145000,95815000,None,-89518000,7000,80941664
1,2022-06-30,USD,215372000,191451000,105537000,105537000,None,20382000,17915000,15174000,...,548000,51710000,112310000,48490000,1121000,92691000,None,-81276000,6000,80473097
2,2022-03-31,USD,198814000,174242000,101101000,101101000,None,21566000,18460000,15584000,...,547000,51847000,67394000,40705000,1028000,84467000,None,-73282000,6000,80147208
3,2021-12-31,USD,183213000,157642000,98398000,98398000,None,21139000,19259000,15930000,...,549000,52067000,67616000,49824000,153000,76615000,None,-65906000,6000,58252727
4,2021-09-30,USD,149887000,123227000,68058000,68058000,None,14432000,20100000,16335000,...,547000,52093000,67640000,65752000,405000,-17116000,None,-44019000,0,36240800


In [47]:
import yfinance as yf
from yahoofinancials import YahooFinancials

In [48]:
yahoo_financials = YahooFinancials('AAPL')
data = yahoo_financials.get_historical_price_data(start_date='2019-01-01', 
                                                  end_date='2019-12-31', 
                                                  time_interval='weekly')
aapl_df = pd.DataFrame(data['AAPL']['prices'])
aapl_df = aapl_df.drop('date', axis=1).set_index('formatted_date')
aapl_df.head()

,high,low,open,close,volume,adjclose
formatted_date,,,,,,
2019-01-01,39.712502,35.500000,38.722500,36.982498,966947200,35.640194
2019-01-08,38.632500,37.130001,37.389999,37.500000,725470000,36.138912
2019-01-15,39.470001,37.512501,37.567501,39.205002,491411200,37.782032
2019-01-22,39.532501,37.924999,39.102501,39.075001,554774800,37.656746
2019-01-29,42.915001,38.527500,39.062500,42.812500,830400800,41.258591


In [52]:
from __future__ import print_function
import time
import intrinio_sdk as intrinio
from intrinio_sdk.rest import ApiException

intrinio.ApiClient().set_api_key('YOUR_API_KEY')
intrinio.ApiClient().allow_retries(True)

identifier = 'AAPL'
page_size = 100
next_page = ''

response = intrinio.ZacksApi().get_zacks_analyst_ratings(identifier=identifier, page_size=page_size, next_page=next_page)
print(response)
    

ModuleNotFoundError: No module named 'intrinio_sdk'

##IGNORE FOR NOW

    NASDAQ DATALINK - FOR SHORT INTEREST

In [28]:
ndl_api = 'dys7Lqv_HziWD1ZFk4ry'

In [29]:
quandl.ApiConfig.api_key = ndl_api

In [30]:
os.chdir('/Users/yash/Desktop/Trading/intern/Basil/finra_short_interest')

In [31]:
df2 = pd.read_csv('equityshortinterest_2023_02_26_05_44_02.csv')

Q4 Earnings 2022

In [12]:
df1.to_csv('us_stocks.csv')

In [39]:
earnings_df = pd.DataFrame(columns=['Symbol', 'Ex-Date', 'title'])

In [40]:
tickers = df1['Ticker']
for t in tickers:
    dft = get_ex_dates(t)
    dft["Date"] = dft["Ex-Date"].apply(
        lambda x: datetime.strptime(x, "%Y-%m-%d")
    )
    dft = dft[dft['Date'] >= datetime(2023,1,1)]
    if dft.shape[0] > 0:
        earnings_df.loc[len(earnings_df.index)] = [t]+list(dft.iloc[0,1:]) 

In [4]:
import csv
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
CSV_URL = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=IBM&interval=1min&slice=year1month1&apikey={av_api}'

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list:
        print(row)

['time', 'open', 'high', 'low', 'close', 'volume']
['2023-03-03 18:34:00', '129.8', '129.8', '129.8', '129.8', '377']
['2023-03-03 18:28:00', '129.76', '129.76', '129.76', '129.76', '179']
['2023-03-03 18:20:00', '129.65', '129.65', '129.65', '129.65', '237']
['2023-03-03 17:12:00', '129.65', '129.65', '129.65', '129.65', '300']
['2023-03-03 17:01:00', '129.65', '129.65', '129.65', '129.65', '500']
['2023-03-03 16:14:00', '129.64', '129.64', '129.64', '129.64', '1225']
['2023-03-03 16:13:00', '129.64', '129.64', '129.64', '129.64', '11428']
['2023-03-03 16:03:00', '129.64', '129.64', '129.64', '129.64', '18975']
['2023-03-03 16:02:00', '129.64', '129.64', '129.64', '129.64', '280']
['2023-03-03 16:01:00', '129.64', '129.64', '129.64', '129.64', '47280']
['2023-03-03 16:00:00', '129.63', '129.685', '129.615', '129.64', '85153']
['2023-03-03 15:59:00', '129.61', '129.63', '129.57', '129.63', '37476']
['2023-03-03 15:58:00', '129.66', '129.66', '129.58', '129.605', '37415']
['2023-03-03 1

In [16]:
import requests
from bs4 import BeautifulSoup

headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:105.0) Gecko/20100101 Firefox/105.0"
}
url = f"https://finance.yahoo.com/quote/MSFT/analysis?p=MSFT"
soup = BeautifulSoup(requests.get(url, headers=headers).content, "html5lib")

for table in soup.select("table"):
    th_row = [th.text for th in table.find_all("th")]
    print(th_row)

    for tr in table.select("tr:has(td)"):
        td_row = [td.text for td in tr.find_all("td")]
        print(td_row)

    print()

['Earnings Estimate', 'Current Qtr. (Mar 2023)', 'Next Qtr. (Jun 2023)', 'Current Year (2023)', 'Next Year (2024)']
['No. of Analysts', '36', '36', '22', '44']
['Avg. Estimate', '2.23', '2.47', '9.35', '10.74']
['Low Estimate', '2.15', '2.14', '9.02', '9.66']
['High Estimate', '2.29', '2.77', '9.72', '11.75']
['Year Ago EPS', '2.22', '2.23', '9.21', '9.35']

['Revenue Estimate', 'Current Qtr. (Mar 2023)', 'Next Qtr. (Jun 2023)', 'Current Year (2023)', 'Next Year (2024)']
['No. of Analysts', '35', '35', '47', '46']
['Avg. Estimate', '51.04B', '54.92B', '208.89B', '231.88B']
['Low Estimate', '50.6B', '52.19B', '206.06B', '215.37B']
['High Estimate', '51.67B', '58.07B', '211.99B', '241.41B']
['Year Ago Sales', '49.36B', '51.87B', '198.27B', '208.89B']
['Sales Growth (year/est)', '3.40%', '5.90%', '5.40%', '11.00%']

['Earnings History', '3/30/2022', '6/29/2022', '9/29/2022', '12/30/2022']
['EPS Est.', '2.18', '2.29', '2.3', '2.29']
['EPS Actual', '2.22', '2.23', '2.35', '2.32']
['Differen

In [ ]:
df2,e = ek.get_data('GOOGL.O',['TR.RevenueActValue.date','TR.RevenueActValue','TR.EPSActValue'],parameters = {'SDate':'0','EDate':'-5','Period':'FQ0','Frq':'FQ'}) 